# Traffic Sign Recognition Classifier

## Deep Learning

## Self-Driving Car Engineering project 2

---
## Step 0: Load The Data

After downloading the training set, unzip the compressed file and move test.p and train.p in the main directory of your project.
The training set contains more then 50000 images labelled through more then 40 classes. The images was resized 32x32

In [ ]:
# Load pickled data
import pickle

#You can change the path if you prefer change the training set files in another directory
training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. 

### Stats about the dataset

In [ ]:
import numpy as np

# Number of training examples
n_train = len(X_train)

# Number of testing examples.
n_test = len(y_train)

# The shape of an traffic sign image
image_shape = X_train[0].shape

# Number of classes
n_classes = 43

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Display images from the dataset

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.

# I created a code that search and display traffic signs of each <<distinct>> class
# Display viz could be improved
import random
import numpy as np
import matplotlib.pyplot as plt 
plt.rcParams.update({'figure.max_open_warning': 0})
%matplotlib inline


sign_list = []
while len(sign_list) < 43:
    index = random.randint(0, len(X_train))
    if y_train[index] not in sign_list:
        sign_list.append(y_train[index])
        image = X_train[index].squeeze()
        plt.figure(figsize=(1,1))
        plt.imshow(image)

----

## Step 2: Design and Test a Model Architecture


### Implementation
#### Data preprocessing

In [ ]:

#The data was preprocessed using sklearn.utils.shuffle that proceeds to a random
#permutation of X_train and y_train elements. By randomising the order of the training set, 
#this process keeps us from using an ordered training set that could affect our algorithm learning.
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
### We split the data into training/validation/testing sets

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X_train,y_train, test_size=0.2,random_state=0)

### Architecture Definition

In [ ]:
import tensorflow as tf
 
EPOCHS = 10
BATCH_SIZE = 200

from tensorflow.contrib.layers import flatten

def LeNet(x):    
    mu = 0
    sigma = 0.1
    
#Part 1
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
#Part 2
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

#Part 3
    fc0   = flatten(conv2)
    
#Part 4
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    fc1    = tf.nn.relu(fc1)

#Part 5
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    fc2    = tf.nn.relu(fc2)

#Part 6
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits


# x is a placeholder to store input batchs. init to None to accept any batc in any size
x = tf.placeholder(tf.float32, (None, 32, 32, 3))

# labels are'nt one hot encoded yet !
y = tf.placeholder(tf.int32, (None))

# One hot encode the labels
one_hot_y = tf.one_hot(y, 43)

**Architecture details:**

**Part 1:**
As suggested, we used LeNet architecture in order to implement our convolutional neural network. We started by building our first convolutional layer that output into 5X5X3 dimension filters with a depth of 6. In this cas we used a stride of 1 and a padding equal to 0 (refers to padding = padding='VALID'). Then we used a ReLU activation fonction where its output will be propagated to a pooling function (Max Pooling in our case) the Convolution layer output using 2x2 Kernal and 2x2 strides.

**Part 2:**
This the block 2, we continue the pipeline with a second convolutional layer that will proceed in the same way as the 1st one. It will use the 1st layer output as a parameter that will be multiplied by the second layer weights in order to calculate the output height and width. Like the 1st layer, we used a ReLU activation function and a Max Pooling function.

**Part 3:**
We make the output of the previous pipeline flat. This mean that we transform the 3D input data to an 1 dimension vector.

**Part 4:**
The previous pipeline output was propagated into a the 3rd layer which is fully Connected. At that level we multiply the received inputs by the weights and we adds the bias. Then we apply a ReLU activation function.

**Part 5:**
Same process than the previous bloc. This layer will receive even less input features.

**Part 6:**
Same thing in the bloc 6. We end the whole process by obtaining the logits that will demonstrates the predicted label performed by our model.

### We train the model

In [ ]:
#Hyperparameter "rate"
rate = 0.001

logits = LeNet(x)

#Part 1
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)


#Part 2
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    #Batch the dataset and run it through the evaluation pipeline
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        #Average the accuracy of each batch
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


#Training execution
with tf.Session() as sess:
    #We initialize the variables
    sess.run(tf.initialize_all_variables())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        #in each epoch we shuffle our training data
        X_train, y_train = shuffle(X_train, y_train)
        
        #Put the training data by batch
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            #Train the model in each batch
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        
        #Each epoch we evaluate the model
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
    #In the end we save the model    
    saver.save(sess, 'lenet')
    print("Model saved")

    
#Evaluate the model on a Test dataset
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### We select new images (non included in the training or the test dataset

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
from os import listdir
from PIL import Image as PImage                                                            
import numpy                                                                     
import matplotlib.pyplot as plt                                                  
import glob

%matplotlib inline
imageFolderPath = 'img/'
im_array = np.array([])
imagesList = []

imagesList = listdir(imageFolderPath)
nbr_examples = len(imagesList)

print('here is a list of new images:')
i=0
while i<len(imagesList):
    im = PImage.open(imageFolderPath+imagesList[i])
    plt.figure(figsize=(1,1))
    plt.imshow(im)
    im_array = numpy.array([numpy.array(PImage.open(imageFolderPath+imagesList[i]))])
    i = i+1

### We use the model to classify the new images

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    #print ("Model restored.")

    #Test for few samples:
    print('Let us see how much our model is able to classify them correctly:')
    ex1 = numpy.array([numpy.array(PImage.open(imageFolderPath+imagesList[1]))])
    ex2 = numpy.array([numpy.array(PImage.open(imageFolderPath+imagesList[6]))])
    ex3 = numpy.array([numpy.array(PImage.open(imageFolderPath+imagesList[4]))])
    ex4 = numpy.array([numpy.array(PImage.open(imageFolderPath+imagesList[2]))])
    ex5 = numpy.array([numpy.array(PImage.open(imageFolderPath+imagesList[3]))])
    
    ex1 = ex1.astype(numpy.float32, copy=False)
    ex2 = ex2.astype(numpy.float32, copy=False)
    ex3 = ex3.astype(numpy.float32, copy=False)
    ex4 = ex4.astype(numpy.float32, copy=False)
    ex5 = ex5.astype(numpy.float32, copy=False)
    
    test_accuracy1 = evaluate(ex1, y_test)
    print("Test Accuracy for the example 1= {:.3f}".format(test_accuracy1))
    test_accuracy2 = evaluate(ex2, y_test)
    print("Test Accuracy for the example 2= {:.3f}".format(test_accuracy2))
    test_accuracy3 = evaluate(ex3, y_test)
    print("Test Accuracy for the example 3= {:.3f}".format(test_accuracy3))
    test_accuracy4 = evaluate(ex4, y_test)
    print("Test Accuracy for the example 4= {:.3f}".format(test_accuracy4))
    test_accuracy5 = evaluate(ex5, y_test)
    print("Test Accuracy for the example 5= {:.3f}".format(test_accuracy5))
    test_accuracy5 = evaluate(ex5, y_test)
    
    
    avr_accuracy_new = (test_accuracy1 + test_accuracy2 + test_accuracy3 + test_accuracy4 + test_accuracy5) / 5
    diva_accuracy = test_accuracy-avr_accuracy_new
    
    print("Accuracy for the 5 images= {:.3f}".format(average_acc))

In [ ]:
# We Visualize the softmax probabilities here.

numpy.set_printoptions(threshold=numpy.nan)

with tf.Session() as sess:
    
    a = tf.constant(ex3)
    b = LeNet(ex3)
    print(sess.run(tf.nn.softmax(a)))
    print(sess.run(tf.nn.top_k(a, k=3)))